# Project: A/B Testing for ShoeFly.com

*This is a project proposed by Codecademy (www.codecademy.com) - course "Data Science", chapter "Aggregates in Pandas"*

Our favorite online shoe store, ShoeFly.com is performing an A/B Test. They have two different versions of an ad, which they have placed in emails, as well as in banner ads on Facebook, Twitter, and Google. They want to know how the two ads are performing on each of the different platforms on each day of the week. Help them analyze the data using aggregate measures.

## Analyzing Ad Sources:

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

ad_clicks = pd.read_csv('ad_clicks.csv')

print(ad_clicks.head())

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google   2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook    7 - Sunday   

  ad_click_timestamp experimental_group  
0               7:18                  A  
1                NaN                  B  
2                NaN                  A  
3                NaN                  B  
4                NaN                  B  


Your manager wants to know which ad platform is getting you the most views. How many views (i.e., rows of the table) came from each utm_source?

In [2]:
total_views_by_source = ad_clicks.groupby('utm_source').user_id.count().reset_index()

print(total_views_by_source)

  utm_source  user_id
0      email      255
1   facebook      504
2     google      680
3    twitter      215


If the column ad_click_timestamp is not null, then someone actually clicked on the ad that was displayed.
Create a new column called is_click, which is True if ad_click_timestamp is not null and False otherwise.

In [3]:
ad_clicks['is_click'] = ~ad_clicks.ad_click_timestamp.isnull()

print(ad_clicks.head())

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google   2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook    7 - Sunday   

  ad_click_timestamp experimental_group  is_click  
0               7:18                  A      True  
1                NaN                  B     False  
2                NaN                  A     False  
3                NaN                  B     False  
4                NaN                  B     False  


We want to know the percent of people who clicked on ads from each utm_source.
Start by grouping by utm_source and is_click and counting the number of user_id‘s in each of those groups. Save your answer to the variable clicks_by_source.

In [4]:
clicks_by_source = ad_clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()

print(clicks_by_source)

  utm_source  is_click  user_id
0      email     False      175
1      email      True       80
2   facebook     False      324
3   facebook      True      180
4     google     False      441
5     google      True      239
6    twitter     False      149
7    twitter      True       66


Now let’s pivot the data so that the columns are is_click (either True or False), the index is utm_source, and the values are user_id.
Save your results to the variable clicks_pivot.

In [6]:
clicks_pivot = clicks_by_source.pivot(columns = 'is_click',
  index = 'utm_source',
  values = 'user_id')

Create a new column in clicks_pivot called percent_clicked which is equal to the percent of users who clicked on the ad from each utm_source.
Was there a difference in click rates for each source?

In [7]:
clicks_pivot['percent_clicked'] = clicks_pivot[True] / (clicks_pivot[True] + clicks_pivot[False])

print(clicks_pivot)

is_click    False  True  percent_clicked
utm_source                              
email         175    80         0.313725
facebook      324   180         0.357143
google        441   239         0.351471
twitter       149    66         0.306977


## Analyzing an A/B Test:

The column experimental_group tells us whether the user was shown Ad A or Ad B.
Were approximately the same number of people shown both adds?

In [9]:
groupby_expgroup = ad_clicks.groupby('experimental_group').user_id.count().reset_index()

print(groupby_expgroup)

  experimental_group  user_id
0                  A      827
1                  B      827


Using the column is_click that we defined earlier, check to see if a greater percentage of users clicked on Ad A or Ad B.

In [12]:
groupby_expgroup = ad_clicks.groupby(['experimental_group', 'is_click']).user_id.count().reset_index()

clicks_pivot_abtest = groupby_expgroup.pivot(columns = 'is_click',
  index = 'experimental_group',
  values = 'user_id')

clicks_pivot_abtest['percent_clicked'] = clicks_pivot_abtest[True] / (clicks_pivot_abtest[True] + clicks_pivot_abtest[False])

print(clicks_pivot_abtest)

is_click            False  True  percent_clicked
experimental_group                              
A                     517   310         0.374849
B                     572   255         0.308343


The Product Manager for the A/B test thinks that the clicks might have changed by day of the week.
Start by creating two DataFrames: a_clicks and b_clicks, which contain only the results for A group and B group, respectively.

In [13]:
a_clicks = ad_clicks[ad_clicks.experimental_group == 'A']
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B']

For each group (a_clicks and b_clicks), calculate the percent of users who clicked on the ad by day.

In [15]:
#Group A:
groupby_a = a_clicks.groupby(['is_click', 'day']).user_id.count().reset_index()

clicks_atest = groupby_a.pivot(columns = 'is_click',
  index = 'day',
  values = 'user_id')

clicks_atest['percent_clicked'] = clicks_atest[True] / (clicks_atest[True] + clicks_atest[False])

print(clicks_atest)

is_click       False  True  percent_clicked
day                                        
1 - Monday        70    43         0.380531
2 - Tuesday       76    43         0.361345
3 - Wednesday     86    38         0.306452
4 - Thursday      69    47         0.405172
5 - Friday        77    51         0.398438
6 - Saturday      73    45         0.381356
7 - Sunday        66    43         0.394495


In [16]:
#Group B:
groupby_b = b_clicks.groupby(['is_click', 'day']).user_id.count().reset_index()

clicks_btest = groupby_b.pivot(columns = 'is_click',
  index = 'day',
  values = 'user_id')

clicks_btest['percent_clicked'] = clicks_btest[True] / (clicks_btest[True] + clicks_btest[False])

print(clicks_btest)

is_click       False  True  percent_clicked
day                                        
1 - Monday        81    32         0.283186
2 - Tuesday       74    45         0.378151
3 - Wednesday     89    35         0.282258
4 - Thursday      87    29         0.250000
5 - Friday        90    38         0.296875
6 - Saturday      76    42         0.355932
7 - Sunday        75    34         0.311927


Based on the results above, we can see that overall the A test had better results, being the ad that I would recommend my company to use.